In [1]:
from time import sleep
import json
import gc
import pandas as pd
import os

In [ ]:
# Setup Selenium pro Colab
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [9]:
def coletar_dados_oceans14():
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.set_window_size(1050, 1010)
    link_oceans_base = 'https://www.oceans14.com.br/acoes/'
    driver.get(link_oceans_base)

    logar_oceans14(driver)
    
    lista_as = driver.find_elements_by_css_selector('table#t1 tbody tr td a')
    dict_ativos = {}
    for a in lista_as:
        link = a.get_attribute('href')
        tick = link.split('/')[-2]

        dict_ativos[tick] = {'link': link}
    
    lista_ticks_buscar = []
    
    print('Entrando por empresa')
    ticks_vazios = {}
    for tick in dict_ativos:
        if(not tick+'.csv' in lista_ticks_buscar):
            dados_tick = {}
            link = dict_ativos[tick]['link']
            driver.get(link)
            driver.implicitly_wait(5)

            try:
                #tick_atual = driver.find_element_by_css_selector('ul#ctl00_conteudoPrincipal_tabsIndicadores li:nth-child(1) a h4').text
                tick_atual = driver.find_element_by_xpath('/html/body/form/div[4]/section/div/div[1]/div/div[4]/div[4]/div/div[2]/div/div/div/ul/li[1]/a/h4').text
            except:
                tick_atual = tick.upper()
            print('   Coletando: '+str(tick_atual))

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(3) td')
            #dict_ativos[tick]['P_L'] = [td.text for td in list_aux]
            dados_tick['P_L'] = [td.text for td in list_aux]

            anos_validos = len(dados_tick['P_L'])
            
            list_aux = driver.find_elements_by_css_selector('div#tabelaLucratividade div table tbody tr:nth-child(1) td')
            list_aux = list_aux[-anos_validos:]
            #dict_ativos[tick]['anos'] = [td.text for td in list_aux]
            dados_tick['ano'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaLucratividade div table tbody tr:nth-child(13) td')
            list_aux = list_aux[-anos_validos:]
            #dict_ativos[tick]['margem_liquida'] = [td.text for td in list_aux]
            dados_tick['margem_liq'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaDivida div table tbody tr:nth-child(6) td')
            list_aux = list_aux[-anos_validos:]
            #dict_ativos[tick]['div_liquida_EBITDA'] = [td.text for td in list_aux]
            dados_tick['divida_liq_EBITDA'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaLiquidez div table tbody tr:nth-child(2) td')
            list_aux = list_aux[-anos_validos:]
            #dict_ativos[tick]['liq_corrente'] = [td.text for td in list_aux]
            dados_tick['liq_corrente'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaLiquidez div table tbody tr:nth-child(3) td')
            list_aux = list_aux[-anos_validos:]
            #dict_ativos[tick]['liq_imediata'] = [td.text for td in list_aux]
            dados_tick['liq_imediata'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(2) td')
            #dict_ativos[tick]['LPA'] = [td.text for td in list_aux]
            dados_tick['LPA'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(4) td')
            #dict_ativos[tick]['VPA'] = [td.text for td in list_aux]
            dados_tick['VPA'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(5) td')
            #dict_ativos[tick]['P_VP'] = [td.text for td in list_aux]
            dados_tick['P_VP'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(6) td')
            #dict_ativos[tick]['P_EBIT'] = [td.text for td in list_aux]
            dados_tick['P_EBIT'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(7) td')
            #dict_ativos[tick]['P_EBITDA'] = [td.text for td in list_aux]
            dados_tick['P_EBITDA'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(8) td')
            #dict_ativos[tick]['P_Ativos'] = [td.text for td in list_aux]
            dados_tick['P_ativos'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(10) td')
            #dict_ativos[tick]['margem_EBITDA'] = [td.text for td in list_aux]
            dados_tick['margem_EBITDA'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(11) td')
            #dict_ativos[tick]['PSR'] = [td.text for td in list_aux]
            dados_tick['PSR'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(12) td')
            #dict_ativos[tick]['ROE'] = [td.text for td in list_aux]
            dados_tick['ROE'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(13) td')
            #dict_ativos[tick]['ROA'] = [td.text for td in list_aux]
            dados_tick['ROA'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(14) td')
            #dict_ativos[tick]['ROIC'] = [td.text for td in list_aux]
            dados_tick['ROIC'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(15) td')
            #dict_ativos[tick]['EV_EBIT'] = [td.text for td in list_aux]
            dados_tick['EV_EBIT'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(16) td')
            #dict_ativos[tick]['EV_EBITDA'] = [td.text for td in list_aux]
            dados_tick['EV_EBITDA'] = [td.text for td in list_aux]

            list_aux = driver.find_elements_by_css_selector('div#tabelaIndicadores01 div table tbody tr:nth-child(17) td')
            #dict_ativos[tick]['DY'] = [td.text for td in list_aux]
            dados_tick['DY'] = [td.text for td in list_aux]

            try:
                if(dados_tick['DY']):
                    df_tick = pd.DataFrame(dados_tick)
                    df_tick.to_csv('dados_oceans14/'+str(tick_atual)+'.csv', encoding='utf-8', index=False)
                else:
                    ticks_vazios[tick_atual] = link
            except:
                ticks_vazios[tick_atual] = link

            a=1
            gc.collect()
    
    json_ativos = json.dumps(ticks_vazios)
    print('----- JSON vazios -----')
    with open('ticks_vazios.json', 'w') as file:
        json.dump(json_ativos, file)
    

    driver.quit()

In [10]:
def logar_oceans14(driver):
    print('--- LOGIN ---')
    usuario = 'pomarox656@gilfun.com'
    usuario = 'hafip75366@tmail7.com'
    senha = 'senhamagica'

    txt_usuario = driver.find_element_by_css_selector('input#txtUsuario')
    txt_senha = driver.find_element_by_css_selector('input#txtSenha')

    txt_usuario.send_keys(usuario)
    sleep(1)
    txt_senha.send_keys(senha)
    sleep(1)

    driver.find_element_by_css_selector('button#loginok').click()
    sleep(1)

In [ ]:
coletar_dados_oceans14()

In [ ]:
!zip -r /content/dados_coletados.zip /content/dados_oceans14